## Загрузка данных

In [129]:
data = []
with open("./smsspamcollection/SMSSpamCollection") as f:
    data = [line.split("\t") for line in f.readlines()]

In [130]:
texts = [line[1] for line in data]
tags = [int(line[0]=="spam") for line in data]

In [131]:
print("общее кол-ва документов:", len(texts))
print("кол-во спама:", sum(tags))

общее кол-ва документов: 5574
кол-во спама: 747


In [132]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np


In [133]:
def build_evaluate(vectorizer, cls):
    X = vectorizer.fit_transform(texts)
    res = cross_val_score(cls, X, tags, scoring="f1", cv=10)
    return np.mean(res)

## Простой Bag of words

In [134]:
res = build_evaluate(CountVectorizer(), LogisticRegression())
print("default CountVectorizer + LogisticRegression: ", res)

default CountVectorizer + LogisticRegression:  0.933348526858


## Примеры прогнозов

In [135]:
text_to_predict = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                   "FreeMsg: Txt: claim your reward of 3 hours talk time",
                   "Have you visited the last lecture on physics?",
                   "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
                   "Only 99$"]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

cls = LogisticRegression()
cls.fit(X, tags)

sample = vectorizer.transform(text_to_predict)
predicted_tags = cls.predict(sample)
print(" ".join(map(str, predicted_tags)))

1 1 0 0 0


## N-граммы + LogisticRegression

In [136]:
ngram_ranges = [(2,2),(3,3),(1,3)]

results = [round(build_evaluate(CountVectorizer(ngram_range=ngram_range), LogisticRegression()),2) for ngram_range in ngram_ranges]
print(" ".join(map(str, results)))

0.82 0.73 0.93


## N-граммы + MultinomialNB

In [138]:
from sklearn.naive_bayes import MultinomialNB

results = [round(build_evaluate(CountVectorizer(ngram_range=ngram_range), MultinomialNB()),2) for ngram_range in ngram_ranges]
print(" ".join(map(str, results)))

0.65 0.38 0.89


## Tf*Idf

In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer

res = build_evaluate(TfidfVectorizer(), LogisticRegression())
print(res)

0.852859955417


Результат получился похуже, чем тот, что был получен с помощью CountVectorizer.  
Но размер выборки и самих текстов небольшой - сильно влияние компонента idf, который одинаков для выбранного слова в разных текстах.

## Эксперименты

In [140]:
for c in range(40, 100, 10):
    for feat_count in [5000, 6000,10000 ]:
        for min_df in [ 0.0009, 0.001, 0.002]:
            for max_df in [0.09, 0.1, 0.2]:
                tfidf = TfidfVectorizer(max_features=feat,min_df=min_df,ngram_range=(1,2), max_df=max_df)
                X_tfidf = tfidf.fit_transform(texts)

                cls = LogisticRegression(C=c)

                res = cross_val_score(cls, X_tfidf, tags, scoring="f1", cv=10)
                if np.mean(res) >0.954:
                    print("tf feat_count: " + str(feat_count) + " min: " + str(min_df) + " c: " + str(c) + " max_df: ", max_df)
                    print(np.mean(res))


tf feat_count: 5000 min: 0.0009 c: 40 max_df:  0.1
0.954236848917
tf feat_count: 5000 min: 0.001 c: 40 max_df:  0.1
0.954236848917
tf feat_count: 6000 min: 0.0009 c: 40 max_df:  0.1
0.954236848917
tf feat_count: 6000 min: 0.001 c: 40 max_df:  0.1
0.954236848917
tf feat_count: 10000 min: 0.0009 c: 40 max_df:  0.1
0.954236848917
tf feat_count: 10000 min: 0.001 c: 40 max_df:  0.1
0.954236848917
tf feat_count: 5000 min: 0.0009 c: 50 max_df:  0.1
0.955653897563
tf feat_count: 5000 min: 0.001 c: 50 max_df:  0.1
0.955653897563
tf feat_count: 6000 min: 0.0009 c: 50 max_df:  0.1
0.955653897563
tf feat_count: 6000 min: 0.001 c: 50 max_df:  0.1
0.955653897563
tf feat_count: 10000 min: 0.0009 c: 50 max_df:  0.1
0.955653897563
tf feat_count: 10000 min: 0.001 c: 50 max_df:  0.1
0.955653897563
tf feat_count: 5000 min: 0.0009 c: 60 max_df:  0.1
0.955653897563
tf feat_count: 5000 min: 0.001 c: 60 max_df:  0.1
0.955653897563
tf feat_count: 6000 min: 0.0009 c: 60 max_df:  0.1
0.955653897563
tf feat_count

Пробовала еще и с использованием триграмм(вместе с униграммами и биграммами), но статистики для триграмм все же маловато.

Лучший результат 0.955653897563 для feat_count: 6000, min_df: 0.001, c: 50, max_df:  0.1

## Выводы

При использовании одних униграмм качество, полученное с помощью CountVectorizer оказалось выше, чем с помощью TfIdfVectorizer. Но следует учитывать, что выборка небольшая и сами тексты маленькие.

Лучшее значение получено при выборе униграмм и биграмм в качестве для формирования признаков. При этом количество признаков было ограничено.

Уменьшение параметра регуляризации привело к росту качества модели, но такая модель может показать не очень хороший результат на тестовых данных, так как могли переобучиться.